In [2]:
import pandas as pd

In [5]:
tenders_data_path = r"~/Capstone/UWACapstoneG2/data/UpdatedAgainTenders.xlsx"
tenders_data = pd.read_excel(tenders_data_path)

In [28]:
dups = pd.concat(g for _,g in tenders_data.groupby("Reference Number") if len(g) > 1)

In [29]:
tenders_no_dups = tenders_data[["Reference Number", "Contract Title", "Description"]].drop_duplicates()

In [30]:
len(dups)

8461

In [31]:
len(tenders_no_dups)

20625

In [32]:
len(tenders_data)

26475

In [33]:
dups_count = len(dups) - len(dups[["Reference Number", "Contract Title", "Description"]].drop_duplicates())

In [34]:
len(tenders_no_dups) + dups_count

26475

In [51]:
import pickle

class Tender:
    def __init__(self, reference):
        self.reference = reference
        self.file_map = {}
        
    def clean_text(self, text):
    # convert from binary string if needed
        if type(text) == bytes:
            text = text.decode("utf-8")
        text = re.sub("[^a-zA-z0-9.,]", " ", text)
        text = re.sub("\\\\", " ", text) 
        text = re.sub("\s+", " ", text)
        text = re.sub("\.+", ".", text)
        return text   
    
    def add(self, file_name, content):
        if content == None:
            print(f"Warning: None content for ref:{self.reference}, fname:{file_name}")
        else:
            content = self.clean_text(content)
            
        if file_name in self.file_map:
            # hopefully wont happen
            print(f"Warning: duplicate file name added for ref:{self.reference} fname:{file_name}")
        else:
            self.file_map[file_name] = content
    
    def save(self):
        with open(f"{self.reference}.pickle", 'wb') as file_handle:
            pickle.dump(self.file_map, file_handle, protocol=pickle.HIGHEST_PROTOCOL)
           
    @staticmethod
    def __correct_handle(reference):
        if ".pickle" in reference: # assume its a fpath, dont change
            return reference
        else: # try ./ref.pickle
            return f"./{reference}.pickle"
        
    @staticmethod
    def exists(reference):
        return os.path.exists(Tender.__correct_handle(reference))
            
    @staticmethod
    def load(reference):
        if Tender.exists(reference):
            with open(Tender.__correct_handle(reference), 'rb') as file_handle:
                t = Tender(reference)
                t.file_map = pickle.load(file_handle)
                return t
        else:
            return None

In [46]:
import os
mismatch = 0
match = 0
# check how many refs exist in the folder
os.chdir(r"/home/ucc/maxichat/Capstone/data/tender_raw/")
for index, row in tenders_no_dups.iterrows():
    ref = str(row["Reference Number"])
    if not Tender.exists(ref):
        mismatch += 1
    else:
        match += 1

print(mismatch)
print(match)

0
20625


In [62]:
def check_tender_comp(path):
    keys = {}
    count = 0
    for file in os.listdir(path):
        t = Tender.load(file)
        if t != None:
            for key in t.file_map.keys():
                print(key)
                print(t.file_map[key])
                if key != "TITLE" and key != "DESCRIPTION":
                    key = "file"
                if key not in keys:
                    keys[key] = 1
                else:
                    keys[key] += 1
        count += 1
        if count > 100:
            break
    
    return keys

In [63]:
print(check_tender_comp(r"/home/ucc/maxichat/Capstone/data/tender_raw_copy/tender_raw"))

TITLE
Multiple Sites Various Agencies Pilbara Routine Maintenance Pest Control
DESCRIPTION
Multiple Sites Various Agencies Pilbara Routine Maintenance Pest Control
TITLE
Provision of Paediatric Psychology Services for WA Country Health Service Clinical, Developmental and Educational Psychologists 
DESCRIPTION
WA Country Health Service WACHS is seeking establish a panel of Contractors, as part of its service delivery model to support Child Development Services, to supply and deliver paediatric psychology services in regional Western Australia. Paediatric clinical, developmental and educational psychology services form a key part of the comprehensive part of child development services to support families to manage the psycho social and emotional health of children with developmental delay and disability. WACHS intends to engage with external psychologists clinical, developmental, and educational to work with the child and family, and other members of the WACHS child development team and 